In [108]:
import pandas as pd
import numpy as np
import csv
import os 
import xml.etree.ElementTree as ET
import xml.dom.minidom
import zipfile
import glob
import re
import spacy
import time

In [2]:
os.chdir('/Users/veliristimaki/Code/Summer Paper')

In [ ]:
tree = ET.parse("0f1653d8-d6bf-4123-a6b0-e165e941d616.xml")
root = tree.getroot()

In [ ]:
text_data = []
for element in root.iter():
    text = element.text
    if text is not None:
        text_data.append(text.strip())

Here I am printing the raw text removing the xml formatting to see what is contained within the drug label information files. As you can see there are a lot of useless text information that will be uninformative when applying text analysis. I will have to parse through the xml attributes to get the elements of interest.

In [ ]:
for text in text_data:
    print(text)

Here I am setting up the parsing object for the xml files.

In [ ]:
domtree = xml.dom.minidom.parse("0f1653d8-d6bf-4123-a6b0-e165e941d616.xml")
group = domtree.documentElement

Here is a function that extracts all the text from the paragraph tags in the xml file format.

In [3]:
def xml_to_str(filename):
    
    domtree = xml.dom.minidom.parse(filename)
    group = domtree.documentElement
    paragraph_elements = group.getElementsByTagName("paragraph")
    
    text_data = []
    
    for i in range(len(paragraph_elements)):
        #if paragraph_elements[i].firstChild is None:
        #    continue
        #print(paragraph_elements[i].firstChild.nodeValue)
        if paragraph_elements[i].firstChild is not None:
            text_data.append(paragraph_elements[i].firstChild.nodeValue)
    
    #Dropping Elements of text_data that only contain empty spaces and \n
    text_filtered = [i for i in text_data if not all(j in ["\n", " "] for j in i)]
    #Dropping empty space and \n within each element of text_data
    text_within = [i.replace("\n", "").replace("  ", " ").strip() for i in text_filtered if not (i.replace("\n", "").replace("  ", " ").strip() == "")]
    
    text_str = ''.join(text_within)
    
    return text_str


In [ ]:
text_data = xml_to_str("0f1653d8-d6bf-4123-a6b0-e165e941d616.xml")

In [ ]:
text_data

Now that we are able to extract the necessary information from each drug label file, we can move on to the next step. First, there is some pre-processing we have to do. The files that were scraped from DailyMed still need to renamed so that we know what drug each file is associated with. From the scraping script I also generated a dictionary file to match NDC codes with drug label files. I will use this dictionary key to rename the zip files. 

In [6]:
column_types = {
    'NDC': str,
    'filename': str,

}

# Read the CSV file and specify the column types
ref_file = pd.read_csv('/Volumes/TOSHIBA EXT/data/clean data/ref.csv', dtype=column_types)

# Perform operations on the DataFrame
# ...


In [7]:
ref_file

,NDC,filename
0,00228202950,a23063c0-099a-4256-b95f-3a857bbf704b
1,00228305911,c5c2d060-452c-4f76-ad93-2e41fd8f06c2
2,10702010101,1bf37f9f-29cf-4b09-9c81-eb369e35a042
3,13107008305,5b851f67-309d-4ef3-8b4b-ade8908256ea
4,13811070910,c45dc1de-adfa-4b3c-a7dc-cffbc8eac74f
...,...,...
1218,24338003501,1e25f905-6c0b-4b19-a3b6-b2a386afa1c3
1219,65162023509,34de1806-d921-4d2f-8c7f-03625b717a70
1220,53746011005,e0cefc7b-e81c-4e15-89a0-fe71dc94338b
1221,70165010030,33f70f58-c871-42c8-8adb-345caeafefcd


As we can see there are multiple NDC codes related to the some package label. This is because 11-digit NDC codes are associated with a particular package size which is not informative for our purposes. Therefore we can drop some NDC codes from the table. 

In [8]:
print(f"There are {len(ref_file['NDC'].unique())} unique NDC codes")
print(f"There are {len(ref_file['filename'].unique())} drug label files")

There are 1223 unique NDC codes
There are 481 drug label files


In [9]:
grouped_df = ref_file.groupby("filename").agg({'NDC': list})
grouped_df

,NDC
filename,
007bf37f-0e46-426a-ac8c-be63d4b7414c,"[29300035505, 29300035501]"
010905f9-3bcb-4b50-9fe8-a3ad0010f14c,[00054024425]
01db4606-d49c-4b10-a78a-1cf41880a9fb,[31722091701]
01f6690c-1410-42ad-8eb3-e7df32eab3ba,"[42043016103, 42043016003]"
021153ce-fe27-4ed1-8d88-b4157b0ed734,"[71093015504, 71093015604, 71093015606]"
...,...
fcd2b59e-8087-475e-9e6b-911bd846ea96,"[71930002012, 71930001912, 71930001952, 719300..."
fe126254-6ee6-48c3-965c-96e4276a5314,"[58657050016, 58657050004]"
fe874f8a-4b56-4d2b-abf5-8742e2633a3b,[16571067516]


In [10]:
for i in range(len(grouped_df)):
    grouped_df['NDC'][i] = list(grouped_df.iloc[i])[0][0]

In [11]:
grouped_df = grouped_df.reset_index()
grouped_df

,filename,NDC
0,007bf37f-0e46-426a-ac8c-be63d4b7414c,29300035505
1,010905f9-3bcb-4b50-9fe8-a3ad0010f14c,00054024425
2,01db4606-d49c-4b10-a78a-1cf41880a9fb,31722091701
3,01f6690c-1410-42ad-8eb3-e7df32eab3ba,42043016103
4,021153ce-fe27-4ed1-8d88-b4157b0ed734,71093015504
...,...,...
476,fcd2b59e-8087-475e-9e6b-911bd846ea96,71930002012
477,fe126254-6ee6-48c3-965c-96e4276a5314,58657050016
478,fe874f8a-4b56-4d2b-abf5-8742e2633a3b,16571067516
479,ff1630b4-2044-433c-96bc-0bb4ae8d42a0,49884011174


In [12]:
claims_2021 = pd.read_csv('/Volumes/TOSHIBA EXT/data/clean data/NH_MA_data_2021.csv', 
                          dtype={"NDC":str}, 
                          usecols = ["NDC", "NDC_PROD_NAME"])

In [13]:
claims_2021

,NDC,NDC_PROD_NAME
0,00228202950,ALPRAZOLAM
1,00228202950,ALPRAZOLAM
2,00228305911,AMPHETAMINE/DEXTROAMPHETA
3,10702010101,METHYLPHENIDATE HYDROCHLO
4,13107008305,LORAZEPAM
...,...,...
80206,69315090601,LORAZEPAM
80207,69315090601,LORAZEPAM
80208,69315090601,LORAZEPAM
80209,43547040610,CLONAZEPAM


In [14]:
merged_df = pd.merge(grouped_df, claims_2021, on='NDC', how = 'left').drop_duplicates()
merged_df

,filename,NDC,NDC_PROD_NAME
0,007bf37f-0e46-426a-ac8c-be63d4b7414c,29300035505,TRAMADOL HYDROCHLORIDE
10,010905f9-3bcb-4b50-9fe8-a3ad0010f14c,00054024425,CODEINE SULFATE
14,01db4606-d49c-4b10-a78a-1cf41880a9fb,31722091701,OXYCODONE HYDROCHLORIDE
43,01f6690c-1410-42ad-8eb3-e7df32eab3ba,42043016103,MODAFINIL
73,021153ce-fe27-4ed1-8d88-b4157b0ed734,71093015504,ZOLPIDEM TARTRATE
...,...,...,...
29517,fcd2b59e-8087-475e-9e6b-911bd846ea96,71930002012,HYDROCODONE BITARTRATE/AC
29533,fe126254-6ee6-48c3-965c-96e4276a5314,58657050016,CODEINE/GUAIFENESIN
29591,fe874f8a-4b56-4d2b-abf5-8742e2633a3b,16571067516,PHENOBARBITAL
29595,ff1630b4-2044-433c-96bc-0bb4ae8d42a0,49884011174,ALPRAZOLAM ODT


In [15]:
len(merged_df['NDC_PROD_NAME'].unique())

164

In [16]:
grouped_by_prod_name = merged_df.groupby('NDC_PROD_NAME').agg({'filename': list}).reset_index()

In [17]:
print(grouped_by_prod_name.iloc[0][0])
print(grouped_by_prod_name.iloc[0][1])

ACETAMINOPHEN/CODEINE
['02386806-02d4-47e3-9c8b-7810146ee795', '83a536d8-385d-46ed-9cd2-47b7efe96ccf', 'eadc80fa-13c3-471a-a22f-6fe7e1ab290f']


In [18]:
grouped_by_prod_name

,NDC_PROD_NAME,filename
0,ACETAMINOPHEN/CODEINE,"[02386806-02d4-47e3-9c8b-7810146ee795, 83a536d..."
1,ACETAMINOPHEN/CODEINE PHO,"[79377617-686b-4640-b2b8-e1358cf82358, 8bc4108..."
2,ADDERALL,[f22635fe-821d-4cde-aa12-419f8b53db81]
3,ADDERALL XR,[aff45863-ffe1-4d4f-8acf-c7081512a6c0]
4,ADHANSIA XR,[80e15bd0-4d56-75f0-ab5a-885879fc56e9]
...,...,...
159,XYOSTED,[8a3d204c-be26-49e0-8599-0ac12a272e81]
160,ZALEPLON,"[47303145-6788-46bd-b9cc-37b59b145cea, 57a1a11..."
161,ZOLPIDEM TARTRATE,"[021153ce-fe27-4ed1-8d88-b4157b0ed734, 0553e26..."
162,ZOLPIDEM TARTRATE ER,"[0c64bc71-2e7f-4b15-a8e1-acb8de04395f, 1a32885..."


From the output below we can see that for some drugs there are multiple products offered. These products tend to be very similar from one another but may differ in their rate of absorption and metabolism, method of ingestion, and other characteristics. 

In [ ]:
for i in range(len(grouped_by_prod_name)):
    if len(grouped_by_prod_name.iloc[i][1]) > 1:
        print(f"{grouped_by_prod_name.iloc[i][0]} has {len(grouped_by_prod_name.iloc[i][1])} different products")

In [ ]:
fld_path = '/Volumes/TOSHIBA EXT/data/XML'

partial_name = '02386806-02d4-47e3-9c8b-7810146ee795'

zip_file_path = glob.glob(f"{fld_path}/*{partial_name}*.zip")

In [ ]:
zip_file_path[0]

In [ ]:
with zipfile.ZipFile(zip_file_path[0], "r") as zip_ref:
    # Find the XML file within the zip file
    xml_files = [file for file in zip_ref.namelist() if file.endswith('.xml')]

    # Verify that there is only one XML file
    if len(xml_files) == 1:
        xml_file_path = xml_files[0]
        
        # Extract the XML file to a temporary location
        zip_ref.extract(xml_file_path, "/Volumes/TOSHIBA EXT/data/XML_extracted")
        
        print(f"XML file '{xml_file_path}' extracted successfully.")
    else:
        print("There should be only one XML file in the zip.")

In [ ]:
xml_file_path

In [ ]:
extracted_xml_file = os.path.join("/Volumes/TOSHIBA EXT/data/XML_extracted", xml_file_path)
new_file_path = os.path.join("/Volumes/TOSHIBA EXT/data/XML_extracted", "test_extract.xml")
        
# Rename the extracted XML file
os.rename(extracted_xml_file, new_file_path)

In [ ]:
print(extracted_xml_file)

In [20]:
ref_table = grouped_by_prod_name.head()

In [155]:
def extract_xml_files(folder_path, extract_path, ref_table):
    
    start_time = time.time()
    
    drug_path_list = []
    
    for i in range(len(ref_table)):
        drug_name = ref_table.iloc[i][0]
        sanitized_drug_name = re.sub(r'[\/:*?"<>| ]+', '_', drug_name)
        
        for j in range(len(ref_table.iloc[i][1])):
        #print(ref_table.iloc[i][1][j])
            zip_file_path = glob.glob(f"{folder_path}/*{ref_table.iloc[i][1][j]}*.zip")
            if len(zip_file_path) > 0:
                zip_file_path = zip_file_path[0]
            #print(zip_file_path)
            
                with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
                # Find the XML file within the zip file
                    xml_files = [file for file in zip_ref.namelist() if file.endswith('.xml')]

                # Verify that there is only one XML file
                    if len(xml_files) == 1:
                        xml_file_path = xml_files[0]
                        #print(xml_file_path)
                # Extract the XML file to a temporary location
                        zip_ref.extract(xml_file_path, extract_path)

                        extracted_xml_file = os.path.join(extract_path, xml_file_path)
                        #print(extracted_xml_file)
                        new_file_path = os.path.join(extract_path, f"{sanitized_drug_name}_prod_no_{j+1}.xml")
                        #print(new_file_path)
                        # Rename the extracted XML file
                        os.rename(extracted_xml_file, new_file_path)

                        #print(f"XML file '{xml_file_path}' extracted successfully.")
                    #else:
                        #print("There should be only one XML file in the zip.")

                        drug_path_list.append(new_file_path)
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print("Files Extracted to", extract_path)
    print("Execution Time:", execution_time, "seconds")
    
    return drug_path_list

In [81]:
folder_path = '/Volumes/TOSHIBA EXT/data/XML'
extract_path = '/Volumes/TOSHIBA EXT/data/XML_extracted'

drug_path_list = extract_xml_files(folder_path,extract_path,ref_table)

In [82]:
drug_path_list

['/Volumes/TOSHIBA EXT/data/XML_extracted/ACETAMINOPHEN_CODEINE_prod_no_1.xml',
 '/Volumes/TOSHIBA EXT/data/XML_extracted/ACETAMINOPHEN_CODEINE_prod_no_2.xml',
 '/Volumes/TOSHIBA EXT/data/XML_extracted/ACETAMINOPHEN_CODEINE_prod_no_3.xml',
 '/Volumes/TOSHIBA EXT/data/XML_extracted/ACETAMINOPHEN_CODEINE_PHO_prod_no_1.xml',
 '/Volumes/TOSHIBA EXT/data/XML_extracted/ACETAMINOPHEN_CODEINE_PHO_prod_no_2.xml',
 '/Volumes/TOSHIBA EXT/data/XML_extracted/ADDERALL_prod_no_1.xml',
 '/Volumes/TOSHIBA EXT/data/XML_extracted/ADDERALL_XR_prod_no_1.xml',
 '/Volumes/TOSHIBA EXT/data/XML_extracted/ADHANSIA_XR_prod_no_1.xml']

In [83]:
str_dict = {}

for i in drug_path_list:
    result = xml_to_str(i)
    last_slash_index = i.rfind("/")
    substring = i[last_slash_index + 1:-4]
    str_dict[substring] = result

In [26]:
nlp = spacy.load('en_core_web_lg')

In [162]:
def create_similarity_matrix(str_dict):
    
    start_time = time.time()

    similarity_table = pd.DataFrame(columns=list(str_dict.keys()), index=list(str_dict.keys()))
    
    for i, str_i in enumerate(str_dict):
        for j, str_j in enumerate(str_dict):
            if j >= i:
                t1 = nlp(str_dict[str_i])
                t2 = nlp(str_dict[str_j])
                cell = t1.similarity(t2)
                similarity_table.loc[str_i, str_j] = cell
                if j > i:
                    similarity_table.loc[str_j, str_i] = cell

    end_time = time.time()
    execution_time = end_time - start_time
    
    print("Similarity Table Constructed!")
    print("Execution Time:", execution_time, "seconds")
    
    return similarity_table

In [119]:
similarity_table = similarity_table(str_dict)

Execution Time: 137.95320391654968 seconds


In [120]:
similarity_table

,ACETAMINOPHEN_CODEINE_prod_no_1,ACETAMINOPHEN_CODEINE_prod_no_2,ACETAMINOPHEN_CODEINE_prod_no_3,ACETAMINOPHEN_CODEINE_PHO_prod_no_1,ACETAMINOPHEN_CODEINE_PHO_prod_no_2,ADDERALL_prod_no_1,ADDERALL_XR_prod_no_1,ADHANSIA_XR_prod_no_1
ACETAMINOPHEN_CODEINE_prod_no_1,1.0,0.999055,0.999045,0.997812,0.998785,0.99346,0.982884,0.956603
ACETAMINOPHEN_CODEINE_prod_no_2,0.999055,1.0,0.998947,0.996624,0.998639,0.994364,0.985055,0.960626
ACETAMINOPHEN_CODEINE_prod_no_3,0.999045,0.998947,1.0,0.997751,0.999783,0.995076,0.98579,0.961444
ACETAMINOPHEN_CODEINE_PHO_prod_no_1,0.997812,0.996624,0.997751,1.0,0.997629,0.990579,0.979842,0.953112
ACETAMINOPHEN_CODEINE_PHO_prod_no_2,0.998785,0.998639,0.999783,0.997629,1.0,0.995238,0.986966,0.963428
ADDERALL_prod_no_1,0.99346,0.994364,0.995076,0.990579,0.995238,1.0,0.993545,0.975677
ADDERALL_XR_prod_no_1,0.982884,0.985055,0.98579,0.979842,0.986966,0.993545,1.0,0.990673
ADHANSIA_XR_prod_no_1,0.956603,0.960626,0.961444,0.953112,0.963428,0.975677,0.990673,1.0


In [ ]:
for i, drug_i in enumerate(drugs):
    for j, drug_j in enumerate(drugs):

In [164]:
#Here we do all the files

folder_path = '/Volumes/TOSHIBA EXT/data/XML'
extract_path = '/Volumes/TOSHIBA EXT/data/XML_extracted'

drug_list = extract_xml_files(folder_path,extract_path,grouped_by_prod_name)

start_time = time.time()

drug_dict = {}

for i in drug_list:
    result = xml_to_str(i)
    last_slash_index = i.rfind("/")
    substring = i[last_slash_index + 1:-4]
    drug_dict[substring] = result

end_time = time.time()
execution_time = end_time - start_time

print("XML to string complete.")
print("Execution Time:", execution_time, "seconds")

similarity_matrix = create_similarity_matrix(drug_dict)

Files Extracted to /Volumes/TOSHIBA EXT/data/XML_extracted
Execution Time: 17.951224088668823 seconds
XML to string complete.
Execution Time: 41.39194178581238 seconds


KeyboardInterrupt: 